In [2]:
!pip install transformers pandas

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
from geopy.geocoders import Nominatim
import time  # Import time for sleep if needed to respect usage policies


In [15]:
# def extract_locations(input_text):
#     # Load the model and tokenizer
#     tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
#     model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#     nlp = pipeline("ner", model=model, tokenizer=tokenizer)

#     # Extract named entities
#     ner_results = nlp(input_text)

#     # Filter for locations
#     locations = [entity['word'] for entity in ner_results if entity['entity'].startswith('B-LOC') or entity['entity'].startswith('I-LOC')]
#     return locations

# def extract_locations(input_text):
#     # Load the model and tokenizer
#     tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
#     model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#     nlp = pipeline("ner", model=model, tokenizer=tokenizer)

#     # Extract named entities
#     ner_results = nlp(input_text)

#     # Initialize variables to track the highest-scoring B-LOC and its score
#     highest_scoring_b_loc = None
#     highest_score = float('-inf')
#     collecting = False
#     collected_locations = []

#     # Iterate through entities to find the highest-scoring B-LOC
#     for entity in ner_results:
#         if entity['entity'].startswith('B-LOC') and entity['score'] > highest_score:
#             highest_scoring_b_loc = entity
#             highest_score = entity['score']
#             collecting = True
#             collected_locations = [entity['word']]  # Reset with the new highest-scoring B-LOC
#         elif collecting and entity['entity'].startswith('I-LOC'):
#             collected_locations.append(entity['word'])
#         elif collecting and not entity['entity'].startswith('I-LOC'):
#             # Stop collecting when a non-I-LOC entity is encountered after starting
#             break

#     return ' '.join(collected_locations) if collected_locations else None
def extract_locations(input_text):
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)

    # Extract named entities
    ner_results = nlp(input_text)

    # Initialize variables to track the highest-scoring B-LOC and its score
    highest_scoring_b_loc = None
    highest_score = float('-inf')
    collecting = False
    collected_locations = []

    # Iterate through entities to find the highest-scoring B-LOC
    for entity in ner_results:
        if entity['entity'].startswith('B-LOC') and entity['score'] > highest_score:
            highest_scoring_b_loc = entity
            highest_score = entity['score']
            collecting = True
            # Reset with the new highest-scoring B-LOC, directly adding the word
            collected_locations = [entity['word'].replace("##", "")]
        elif collecting and entity['entity'].startswith('I-LOC'):
            # For subsequent I-LOC tokens, append them, removing "##" if present and not adding a space before it
            word = entity['word']
            if word.startswith("##"):
                # Combine with the previous token
                collected_locations[-1] += word.replace("##", "")
            else:
                collected_locations.append(word)
        elif collecting and not entity['entity'].startswith('I-LOC'):
            # Stop collecting when a non-I-LOC entity is encountered after starting
            break

    # Join the collected location tokens into a single string
    location_string = ' '.join(collected_locations)
    return location_string if collected_locations else None


In [16]:

# Define the geolocator with a specific user_agent
geolocator = Nominatim(user_agent="ethan3048V2")

def get_location_coordinates(location_name):
    # Simple wrapper to retry geocoding in case of failure
    try:
        location = geolocator.geocode(location_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            return None
    except Exception as e:
        print(f"Error geocoding {location_name}: {e}")
        return None


In [19]:
# def process_csv(input_csv_path, output_csv_path):
#     # Read the input CSV
#     df = pd.read_csv(input_csv_path, on_bad_lines='skip')

#     # Extract locations for each row in the 'content' column
#     df['Locations'] = df['content'].apply(extract_locations)

#     # Create a new dataframe with 'URL' and 'Locations'
#     output_df = df[['URL', 'Locations']]

#     # Write to a new CSV
#     output_df.to_csv(output_csv_path, index=False)

# def process_csv(input_csv_path, output_csv_path):
#     # Read the input CSV
#     df = pd.read_csv(input_csv_path, on_bad_lines='skip')

#     # df = pd.read_csv(input_csv_path, on_bad_lines='skip', nrows=10) # nrows just for testing
#     # Extract locations for each row in the 'content' column
#     df['Locations'] = df['content'].apply(extract_locations)

#     # Add a new column for coordinates
#     df['Coordinates'] = df['Locations'].apply(lambda loc: get_location_coordinates(loc) if loc else None)

#     # Create a new dataframe with 'URL', 'Locations', and 'Coordinates'
#     output_df = df[['URL', 'Locations', 'Coordinates']]

#     # Write to a new CSV
#     output_df.to_csv(output_csv_path, index=False)

def process_csv(input_csv_path, output_csv_path):
    # Read the input CSV
    df = pd.read_csv(input_csv_path, on_bad_lines='skip')

    # df = pd.read_csv(input_csv_path, on_bad_lines='skip', nrows=10) # Uncomment for testing with the first 10 rows

    # Extract locations for each row in the 'content' column
    df['Locations'] = df['content'].apply(extract_locations)

    # Add a new column for coordinates
    df['Coordinates'] = df['Locations'].apply(lambda loc: get_location_coordinates(loc) if loc else None)

    # No need to create a new DataFrame. The original df now has all the required data.

    # Write the DataFrame (with all original columns plus the new ones) to a new CSV
    df.to_csv(output_csv_path, index=False)


In [ ]:
input_csv_path = './filtered.csv'
output_csv_path = '/content/drive/MyDrive/Capstone Personal/urlAndLocationAndCoordsV2.csv'

process_csv(input_csv_path, output_csv_path)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. ini

Error geocoding Las Vegas: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Las+Vegas&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Nevada: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Nevada&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Las Vegas: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Las+Vegas&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Las Vegas: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Las+Vegas&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


...

Error geocoding Ohio: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Ohio&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Los Angeles: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Los+Angeles&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Michigan: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Michigan&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Illinois: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Illinois&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Michigan: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Michigan&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [14]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# # Load tokenizer, model, and NER pipeline
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# # Your example text
# example = """Reece Rogers With basically a billion streaming services to pick from, it can be hard to know where to watch big events—especially the Super Bowl, which frequently switches networks. No need to fret though. It’ll easy to watch the Kansas City Chiefs play the San Francisco 49ers for free online—no cable subscription required. The two teams go head to head for Super Bowl LVIII in Las Vegas, Nevada today. Are you having friends over for a raucous party to watch the Big Game? Or are you just tuning in to catch Usher’s halftime show and eat some nachos? Either way, here’s everything you need to know about streaming the Super Bowl online. (We also included info on how to catch the Puppy Bowl, an adorable alternative, for those not into sports.) Kickoff for the 2024 Super Bowl is scheduled for 6:30 pm ET today. Can’t wait for the action to begin? You can expect the pregame show to start around 2 pm ET on Paramount+. Wanting to check out the Puppy Bowl? Celebrating 20 years of fluffy sportsmanship, the televised event starts at 2 pm ET, which gives viewers plenty of time to check it out before the main game begins. Want to watch the Super Bowl for free? New users of Paramount+ can stream the game live by signing up for a seven-day trial. (Set a reminder to cancel your subscription after the game if you decide not to stick around.) The basic Paramount+ plan costs $6 a month, and the upgraded plan that has few ads and access to Showtime content costs twice as much. Do you often find yourself missing the feeling of watching live television? Having second thoughts about cord-cutting, but not ready to jump back into that kind of commitment? Consider a monthly subscription to a live TV streaming service. Hulu + Live TV, YouTube TV, and FuboTV are three popular options that all carry CBS. After the introductory periods, these subscriptions cost around $75 a month. It’s likely too late to purchase new gear before the Super Bowl starts, but you might want to consider getting an over-the-air antenna for future situations like this. Using the relatively cheap tool, you can watch local channels, including CBS, without any kind of subscription. The reception quality and channel availability depends on your location, so be sure to test it out before any live events. Where can you stream the Puppy Bowl? If you don’t have cable or a service like YouTube TV, the Puppy Bowl is available to stream with a subscription to Max, which has plans starting at $10 a month. A more niche but cheaper option is to try out Discovery+, a streaming service with tons of trashy reality TV. Discovery+ plans start at $5 a month and include a one-week free trial. Following in the footsteps of Rihanna’s blockbuster, baby-bump revealing performance at last year’s Super Bowl, Usher is the headliner for this year’s halftime show. The eight-time Grammy winner is best known for his energetic club hits, like “DJ Got Us Fallin’ In Love” and “Yeah!” Who knows! Even though Taylor Swift has shown up for select Chiefs games, and boosted ratings, since she started dating Chiefs tight end Travis Kelce, it’s unclear whether the pop star will be in the stands to cheer for him. Swift is performing in Tokyo the week leading up to the Super Bowl. It’s unclear whether she’ll fly back for the game, and tracking her private flights may soon get harder. Carlton Reid Reece Rogers Dell Cameron Steven Levy"""
# # example = """Fox News contributor Joe Concha joins "Fox & Friends First" to discuss President Biden declining a Super Bowl interview for the second year in a row and the liberal media panicking over the latest 2024 polls. Get ready for the Super Bowl this Sunday as the San Francisco 49ers face the Kansas City Chiefs at Allegiant Stadium on the Las Vegas strip. You don’t want to settle for a mediocre TV experience. So, it's time to learn how to improve your current TV setup or upgrade to a new one before game day. CHIEFS' TRAVIS KELCE FOCUSED ON ONLY 1 RING HEADING INTO SUPER BOWL Remote in hand in front of a flat screen (Kurt "CyberGuy" Knutsson) If you’re already content with your current TV and setup and just want to gear up for the game, below are the top five ways to enhance your viewing experience: Services like Paramount+ offer the ability to stream the Super Bowl this year. You’ll want to make sure you don’t lose the game in two ways: Make sure you’re using a wired connection to the internet if possible. That way, you’re not depending on a Wi-Fi signal, which may be impacted by how many people are over at your home for the game and using your Wi-FI for their devices or who else is streaming elsewhere in your home. On that note, make sure you’re maximizing bandwidth by making sure your kids or spouse isn’t streaming in higher resolutions while the game is on. MORE: BEST WI-FI RANGE EXTENDERS Paramount+ promotion (Paramount) The game is the most popular thing to watch that Sunday, so if you’ve got spotty coverage and are concerned about drops in service, you should have a way to broadcast the game traditionally (aka "over the air"), so keep an antenna ready. While the picture quality often steals the show when it comes to watching the Super Bowl, sound quality can enhance or diminish the experience. Most TVs these days have audio settings, which, if adjusted, can really enhance the game experience in your home. If you’re more interested in the commentary, then you can adjust the sound to boost the dialogue. In most TVs, you can usually find the audio presets and adjustments under "Setting," then "Audio" or "Sound." There, you can play around with the treble, bass and equalizer. Some of the presets, such as "movie," enhance dialogue or "standard" to level out the commentary. Sometimes, it is worth checking to make sure the bass is at optimal levels, as having it too high can cause distortion. On the other hand, if you prefer to hear every impact of every play, you can turn on the bass ‘just so’ to add to the drama. TV showing setting adjustments for sound (Kurt "CyberGuy" Knutsson) MORE: TOP 10 SOUNDBARS TO ENHANCE YOUR TV AUDIO EXPERIENCE To make sure that friends behind those huddled in front of the TV can enjoy the game just as much, check the picture quality of your set. For LEDs, if it seems too dark or dim, turn up the backlight control, which can increase the brightness behind the LCD screen. For OLEDs, instead of just turning up the brightness, turn up the OLED light setting. Additionally, make sure you turn off the energy saver or auto-brightness control settings before game day, as it can impact screen brightness. TV showing settings for the picture quality (Kurt "CyberGuy" Knutsson) Some TVs come with smoothing or dejudder settings, sometimes referred to as the "Soap Opera effect." It can be helpful to adjust this setting or turn it off or on, depending on your preference, before the game. On one hand, it could smooth out motions made on the field or create too much distraction by creating graphical trails from fast motions. Auto Motion Plus Settings on a TV (Samsung) If you were just waiting for an excuse to upgrade your TV setup for game day, delay no more. Here are five things to look for in a new TV. 1) Screen size and resolution: You want a TV that can display the game in high definition and has a large enough screen to enjoy the action from any angle. Look for TVs that have at least 4K resolution (3840 x 2160 pixels) and a screen size of 55 inches or more. Some TVs also offer 8K resolution (7680 x 4320 pixels) for even more detail, but they are more expensive and not widely available. 2) Refresh rate and motion handling: You want a TV that can handle fast-moving scenes without blurring or stuttering. Look for TVs that have a high refresh rate (the number of times the screen updates per second) and a low input lag (the time it takes for the TV to respond to your commands). A refresh rate of 120 Hz or more and an input lag of 15 ms or less are ideal for gaming and sports. Some TVs also have features like variable refresh rate (VRR) and auto low latency mode (ALLM) that can adjust the refresh rate and input lag dynamically according to the content. CLICK HERE FOR MORE SPORTS COVERAGE ON FOXNEWS.COM 3) HDR and color performance: You want a TV that can deliver vivid and realistic colors and contrast. Look for TVs that support high dynamic range (HDR) formats like HDR10, Dolby Vision, or HDR10+. These formats can enhance the brightness, contrast, and color range of the image. Also, look for TVs that have a wide color gamut (the range of colors the TV can produce) and high color accuracy (how close the colors are to the original source). 4) Sound quality and features: You want a TV that can produce clear and immersive sound. Look for TVs that have a good sound system with multiple speakers and a subwoofer. Some TVs also have features like Dolby Atmos or DTS:X that can create a surround sound effect with overhead and directional audio. Alternatively, you can also connect your TV to a soundbar or a home theater system for a better audio experience. 5) Smart features and connectivity: You want a TV that can stream your favorite apps and services and connect to your other devices. Look for TVs that have a smart platform like Android TV, Roku TV, Fire TV, or webOS. These platforms can offer a variety of apps and services like Netflix, Hulu, Amazon Prime Video, Paramount+, Max, Disney+, YouTube and more. Also, look for TVs that have multiple HDMI ports and USB ports to connect your gaming consoles, streaming devices, soundbars, and other peripherals. Some TVs also have features like voice control, AirPlay, Chromecast or Miracast that can let you control your TV or cast content from your smartphone or tablet. Check out how to score a great deal on a TV before the Super Bowl. There is really no bad way to enjoy the big game this year, but if you want to maximize your game viewing experience, there are many ways to enhance your current setup. And, of course, if you need an excuse to splurge, there are some top contenders that fit the bill. If you are splurging, do remember that the cost of these units doesn’t include the cost of installation. Some of these TVs need to be mounted for best viewing, which might incur additional costs. CLICK HERE TO GET THE FOX NEWS APP Do you pay attention to all these details before game day, or do you just turn on the set and get lost in the game no matter what? What are your plans for this year’s Super Bowl game? Let us know by writing us at Cyberguy.com/Contact. For more of my tech tips & security alerts, subscribe to my free CyberGuy Report Newsletter by heading to Cyberguy.com/Newsletter. Ask Kurt a question or let us know what stories you'd like us to cover. Answers to the most asked CyberGuy questions: Copyright 2024 CyberGuy.com. All rights reserved. Kurt "CyberGuy" Knutsson is an award-winning tech journalist who has a deep love of technology, gear and gadgets that make life better with his contributions for Fox News & FOX Business beginning mornings on "FOX & Friends." Got a tech question? Get Kurt’s CyberGuy Newsletter, share your voice, a story idea or comment at CyberGuy.com. Get a daily look at what’s developing in science and technology throughout the world. Subscribed You've successfully subscribed to this newsletter!"""
# # example = """Heavy downpours Monday caused flooded roadways, car wrecks and water rescues across San Diego County, California. The worst of the storm started in the morning, prompting the state to issue travel warnings across the county. San Diego officials urged residents to stay at home unless travel was absolutely necessary. San Diego Mayor Todd Gloria wrote in a post on X, formerly known as Twitter, that he had declared a state of emergency in the city due to the extreme rainfall and flash flooding. Gloria advised residents to, "never attempt to travel on flooded roads." Video shows Interstate 5 in downtown San Diego flooded as a driver drove through the high water level. The National Weather Service reposted the video on their page on X issuing a warning for the dangers of driving through flooded roads. Photos and a video recorded by a teacher from Village Elementary School showed the water levels in the hallways and classrooms. A flash flood warning was in effect Monday for parts of North County including; Oceanside, Carlsbad, Vista and Solana Beach. The warning was extended to include Chula Vista, San Diego and El Cajon until 12:45 p.m., the weather service said. The San Diego Airport received 2.73 inches on rain just before midnight on Monday. According to the weather service, Jan. 22, 2024, was the fourth wettest day in the area. The record for the wettest day was set nearly 170 years ago, on Dec. 02, 1854, with 3.34 inches of rain reported. An Encanto resident video taped the flood waters as it swept cars away. The resident, who spoke in Spanish, said, "Look at the cars my God, my God!" San Diego firefighters and lifeguards conducted 24 rescues from the San Diego and Tijuana rivers and hundreds more from homes and cars. The organizations have reported no fatalities from the flooding thus far, according to a post from the San Diego Fire Department on X."""
# # Get NER results
# ner_results = nlp(example)

# # Filter results to only include locations
# location_entities = [entity['word'] for entity in ner_results if entity['entity'].startswith('B-LOC') or entity['entity'].startswith('I-LOC')]

# print(location_entities)
print(extract_locations("""With a healthy dose of heart and whimsy, the Sundance documentary Seeking Mavis Beacon follows two young Black women who are devoted to finding the original model for Mavis Beacon Teaches Typing. If you touched a computer during the '80s or '90s, there's a good chance that Mavis helped you get comfortable with a keyboard. Or at the very least, you might remember her from the program's original 1987 cover: a smiling, elegant Black woman dressed in a cream-colored outfit. She embodied style and professional poise — it was as if you could be just as capable as her if you bought that program. It's no spoiler to say that "Mavis Beacon" didn't really exist – she was a marketing idea crafted by a group of white dudes from Silicon Valley. But the program's cover star was real: Her name was Renee L'Esperance, a Haitian model who was discovered while working at Saks Fifth Avenue in Los Angeles. After her image helped make Mavis Beacon Teaches Typing a success, she retreated from the spotlight, reportedly heading back to retire in the Caribbean. The documentary's director and writer, Jazmin Jones, as well as her collaborator, Olivia McKayla Ross, start with those basic details and set out to find L'Esperance like a pair of digital detectives. From a home base in a rundown Bay Area office – surrounded by tech ephemera, a variety of art pieces and images of influential black women – they lay out L'Esperance's reported timeline, follow leads and even host a spiritual ceremony to try and connect with the model. I won't say if the pair actually end up finding L'Esperance because it's the journey that makes Seeking Mavis Beacon such a joy to watch. Jones and Ross both grew up with the typing program and felt a kinship toward the character of Mavis Beacon. It was the first program to prominently feature a Black woman on the cover (a move that reportedly caused some suppliers to cut their orders), so it made the technology world seem like somewhere young Black women could actually fit in. Beacon's digital hands also appear on-screen, as if she's gently guiding your fingers to the correct letters and positioning. To help uncover more details about the whereabouts of Mavis Beacon, Jones and Ross set up a hotline and website for anyone to submit clues. Some of those calls are featured in the film, and they make it clear that her digital presence inspired many people. The film opens with references to Beacon throughout culture, including one of my favorite bits from Abbott Elementary, where Quinta Brunson's over-achieving teacher is far too excited to spot the typing icon in a school crowd. I was reminded of my own childhood experience with Mavis Beacon Teaches Typing, spending free periods at school and idle time at home trying to get my typing speed up. By middle school, typing felt as natural as breathing. And yes, I would also have freaked out if I saw the real Beacon in person. While the documentary doesn't seem out of place at Sundance, which is known for innovative projects, it also sometimes feels like a piece of experimental media meant for YouTube or an art show filled with impossibly cool twenty-somethings. (At one point, Ross attends a farewell ceremony for one of her friends' dead laptops, which was hosted in an art space filled with people dressed in white. That's the sort of hip weirdness that will either turn you off of this film, or endear you to it more.) Jones shows us screen recordings of her own desktop, where she may be watching a TikTok alongside her notes. Instead of a full-screen video chat with another person, sometimes we just see a FaceTime window (and occasionally that reflects Jones' own image looking at the screen). Finding Mavis Beacon tells its story in a way that digital natives will find natural, without locking itself exclusively into screens like the film Searching. As is true for many first features, the film could use some narrative tightening. Jones and Ross's investigation stalls at several points, and we're often just left adrift as they ponder their next steps. The pair also occasionally appear too close to the story, or at least, that's how it seems when we see Jones tearing up while pleading to meet with L'Esperance. But I'd argue that's also part of the charm of Seeking Mavis Beacon. Jones and Ross aren't some true crime podcast hosts looking to create content out of controversy. They're young women who found comfort in one of the few faces in tech that looked like them. With this film, Jones and Ross could be similarly inspirational for a new generation of underrepresented techies."""))


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Silicon Valley


In [20]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
# def extract_locations(input_text):
#     # Load the model and tokenizer
#     tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
#     model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#     nlp = pipeline("ner", model=model, tokenizer=tokenizer)

#     # Extract named entities
#     ner_results = nlp(input_text)

#     # Initialize variables to track the highest-scoring B-LOC and its score
#     highest_scoring_b_loc = None
#     highest_score = float('-inf')
#     collecting = False
#     collected_locations = []

#     # Iterate through entities to find the highest-scoring B-LOC
#     for entity in ner_results:
#         if entity['entity'].startswith('B-LOC') and entity['score'] > highest_score:
#             highest_scoring_b_loc = entity
#             highest_score = entity['score']
#             collecting = True
#             collected_locations = [entity['word']]  # Reset with the new highest-scoring B-LOC
#         elif collecting and entity['entity'].startswith('I-LOC'):
#             collected_locations.append(entity['word'])
#         elif collecting and not entity['entity'].startswith('I-LOC'):
#             # Stop collecting when a non-I-LOC entity is encountered after starting
#             break

#     return ' '.join(collected_locations) if collected_locations else None

# # Test the function
# input_text = """By Ayana Archie A San Francisco 49ers fan reacts while watching a telecast of NFL football's Super Bowl 58 on a screen outside the Chase Center in San Francisco on Sunday, Feb. 11, 2024. Noah Berger/AP hide caption A San Francisco 49ers fan reacts while watching a telecast of NFL football's Super Bowl 58 on a screen outside the Chase Center in San Francisco on Sunday, Feb. 11, 2024. Sunday's game was the most watched Super Bowl in history. About 123.4 million people tuned in across several platforms, including CBS, Paramount+, Univision and Nickelodeon, which featured commentary from SpongeBob SquarePants and friends. All together, more than 200 million people watched at least some part of the game, according to CBS, which cited Nielsen and Adobe Analytics data. The Kansas City Chiefs became back-to-back Super Bowl champs Sunday after defeating the San Francisco 49ers 25-22 in overtime. Viewership was up 7% from last year's championship game, which at the time held the record for the most watched Super Bowl. Sunday's game also was the most watched television broadcast for a single network, CBS said. Approximately 2.2 million people tuned into the game on Univision, making it the most viewed Spanish language broadcast of the Super Bowl, CBS said. Several factors likely contributed to the high viewership. Earlier this month, Nielsen expanded its availability of "out-of-home" viewership counts – which include audiences at airports, hotels, bars, etc. – to all 50 states. Additionally, the game went into overtime and, as throughout much of the season, Taylor Swift, who is dating Chiefs tight end Travis Kelce, was present at the game."""
# locations = extract_locations(input_text)
# print(locations)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


San Francisco


In [14]:
# from geopy.geocoders import Nominatim

# # Try using a more specific user_agent name
# geolocator = Nominatim(user_agent="ethan3048")

# def get_location_coordinates(location_name):
#     try:
#         location = geolocator.geocode(location_name)
#         if location:
#             return (location.latitude, location.longitude)
#         else:
#             return None, None
#     except Exception as e:
#         print(f"Error geocoding {location_name}: {e}")
#         return None, None

# # Example usage
# location_name = "Eiffel Tower"
# latitude, longitude = get_location_coordinates(location_name)
# if latitude and longitude:
#     print(f"Coordinates of {location_name}: Latitude = {latitude}, Longitude = {longitude}")
# else:
#     print("Location not found or error in geocoding.")


Coordinates of Eiffel Tower: Latitude = 48.8582599, Longitude = 2.2945006358633115
